<a href="https://colab.research.google.com/github/BecomeAllan/S2Search/blob/main/SemanticScholarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consumindo a API do SemanticScholar

A seguir, tem uma classe sem parámetros chamada `Search()`, que ao instanciar-la em uma variável é possível fazer pesquisas sobre papers utilizando a api do SemanticScholar, dentre os parâmetros temos:

- Buscar: Pesquisas sobre tópicos onde adicionar tópicos utiliza-se + (mais) e remover tópicos usamos - (menos)

- Fields: O que será retornado como dados. Para utilizar, escolha dentre as opções sem utilizar espaço e separadas de virgulas:
  - externalIds
  - url
  - title
  - abstract
  - venue 
  - year 
  - referenceCount
  - citationCount
  - influentialCitationCount
  - isOpenAccess
  - fieldsOfStudy
  - authors 

- Offset: Número que começa a puxar a partir da ordem dele a lista de papers. (0 seria o primeiro)

- Limite: Número de papers a ser retornados (Máx. 10.000)

**Obs:** A api do SemanticScholar disponibiliza 100 query's a cada 5 min, no qual apenas retorna no máx. 100 resutados (limite). Assim a cada 5 min, é possível puxar 10.000 papers.

In [227]:
#@title Classe para pesquisa no SemanticScholar
import IPython
from google.colab import output
import pandas as pd

class Search():
  def __init__(self):
    self.data = ""
    self.data_0 = ""

    self._start()
  
  def _start(self):

    output.register_callback('notebook.searching', self._searching)
    output.register_callback('notebook.AddListItem', self._add_list_item)
    output.register_callback('notebook.mergeData', self._merge_data)
    output.register_callback('notebook.error', self._error)

    display(IPython.display.HTML('''
        <label for="query">Buscar: </label>
        <input type="text" id="query" value="Machine Learning+Deep Learning" style="width: 400px;"/>
        <br/>
        <br/>
        
        <label for="fields">Fields: </label>
        <input type="text" id="fields" value="title,abstract,isOpenAccess,fieldsOfStudy" style="width: 400px;"/>
        <br/>
        <br/>
 
        <label for="limit">Limite: </label>
        <input type="text" id="limit" value="10" style="width: 50px;"/><br/>
        <br/>

        <label for="limit">Offset: </label>
        <input type="text" id="offset" value="0" style="width: 50px;"/><br/>
        <br/>

        <button id='button'>Pesquisar</button>
        <br/>
        <br/>
        <script>
          document.querySelector('#button').onclick = async () => {

            var search = document.getElementById("query").value
            var fields = document.getElementById("fields").value
            var limit = parseInt(document.getElementById("limit").value)
            var offset = parseInt(document.getElementById("offset").value)

            google.colab.kernel.invokeFunction('notebook.searching', [], {});

            if (limit >100) {
              var number = limit
              var data = ""
              var promises = []
              var offsetSearch = 0
              var rest = 0

              for (let index = 0; index < Math.floor(limit/100); index++) {
                offsetSearch = 100*(index) + offset + 1*(index!==0)


                promises.push(
                  fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=100&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )
              }
              
              if (limit%100 !== 0) { 
                rest= limit%100
                offsetSearch = offsetSearch+100
                
                console.log(rest)
                console.log(offsetSearch)

                promises.push(
                fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=${rest}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )}

              await Promise.all(promises).then(data=>{
                google.colab.kernel.invokeFunction('notebook.mergeData', [data], {})
              })
              .catch(err=> { return (google.colab.kernel.invokeFunction('notebook.error', [err], {})) })

            } else {
            await fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=${limit}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {
      return(google.colab.kernel.invokeFunction('notebook.AddListItem', [res], {}))})
    .catch(err=> { return (
      google.colab.kernel.invokeFunction('notebook.error', [err], {})) })
            }
      
     // .catch(err=>{
    //   google.colab.kernel.invokeFunction('notebook._error', [], {});
    //   })


          };
        </script>
        '''))
    

  def _error(self,value):
    try:
      print("ERRO na API SemanticScholar:")
      print(value)
    except:
      pass 

  def _searching(self):
    with output.use_tags('some_outputs'):
      print("\n\nPesquisando...")
      sys.stdout.flush();

  def _merge_data(self, data):
    output.clear(output_tags='some_outputs')
    print(f"Achou {data[0]['total']} papers.\n")
    self.data_0 = data

    self.data = pd.DataFrame(data[0]['data'])

    try:
      for x in data[1:len(data)]:
        try:
          self.merge(pd.DataFrame(x['data']))
        except:
          self._error(x)
    except:
      pass 


    print(f"Api devolveu >> {self.data.shape[0]} papers\n" )
    print(self.data.head())


  def merge(self, data):
    self.data = pd.concat([self.data, data], ignore_index=True ) 

  def _add_list_item(self, value):
    output.clear(output_tags='some_outputs')
    print(f"Achou {value['total']} papers.\n")
    print(f"Api devolveu >> {self.data.shape[0]} papers\n" )

    self.data = pd.DataFrame(value['data'])
    
    print(self.data.head())



In [ ]:
Resultados = Search()

In [ ]:
# Os dados ficam na variável data, no qual é uma tabela do tipo pandas
print(Resultados.data)

In [ ]:
Resultados_2 = Search()

In [ ]:
print(Resultados_2.data)

In [ ]:
# Disponibiliza uma função merge para concatenar resultados de outras pesquisas
Resultados_2.merge(Resultados.data)

print(Resultados_2.data)

Resultados_2.data.drop_duplicates("paperId")